In [2]:
import anthropic

# Loads variables from the environment
from os import getenv
from dotenv import load_dotenv
load_dotenv()
open_api_key = getenv("ANTHROPIC_API_KEY")

# Print output of message type
import json
def print_msg(message):
    print(json.dumps(message.model_dump(), indent=2))


### Basic request and response

In [3]:
message = anthropic.Anthropic().messages.create(
    model="claude-opus-4-6",
    max_tokens=1024,
    messages=[{"role": "user", "content": "Hello, Claude"}],
)
print_msg(message)

{
  "id": "msg_01BhRxTHzky9WCALum24Wf4N",
  "content": [
    {
      "citations": null,
      "text": "Hello! How are you doing today? Is there something I can help you with?",
      "type": "text"
    }
  ],
  "model": "claude-opus-4-6",
  "role": "assistant",
  "stop_reason": "end_turn",
  "stop_sequence": null,
  "type": "message",
  "usage": {
    "cache_creation": {
      "ephemeral_1h_input_tokens": 0,
      "ephemeral_5m_input_tokens": 0
    },
    "cache_creation_input_tokens": 0,
    "cache_read_input_tokens": 0,
    "inference_geo": "global",
    "input_tokens": 10,
    "output_tokens": 20,
    "server_tool_use": null,
    "service_tier": "standard"
  }
}


### Multiple conversational turns

The Messages API is stateless, which means that you always send the full conversational history to the API. You can use this pattern to build up a conversation over time. Earlier conversational turns don't necessarily need to actually originate from Claude — you can use synthetic `assistant` messages.

In [4]:
message = anthropic.Anthropic().messages.create(
    model="claude-opus-4-6",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": "Hello, Claude"},
        {"role": "assistant", "content": "Hello!"},
        {"role": "user", "content": "Can you describe LLMs to me?"},
    ],
)
print_msg(message)

{
  "id": "msg_018y2rnr8Bf2Tuqk3KWck5N3",
  "content": [
    {
      "citations": null,
      "text": "# Large Language Models (LLMs)\n\nLarge Language Models are AI systems trained to understand and generate human language. Here's a breakdown:\n\n## What They Are\n- Neural networks with **billions of parameters** (adjustable weights)\n- Trained on massive amounts of text data from books, websites, articles, and other sources\n- Designed to predict and generate text based on patterns learned during training\n\n## How They Work\n- At their core, they predict **the next most likely word (or token)** in a sequence\n- They use a architecture called a **Transformer**, which helps them understand context and relationships between words\n- They process text as \"tokens\" \u2014 chunks of words or characters\n\n## Key Capabilities\n- Answering questions\n- Writing and summarizing text\n- Translation\n- Coding assistance\n- Reasoning and analysis\n- Conversation\n\n## Limitations\n- They can **

### Vision

Claude can read both text and images in requests. We support both `base64` and `url` source types for images, and the `image/jpeg`, `image/png`, `image/gif`, and `image/webp` media types. See our [vision guide](https://platform.claude.com/docs/en/build-with-claude/vision) for more details.

In [5]:
import base64
import httpx

##### Option 1

In [7]:
# Option 1: Base64-encoded image
image_url = "https://upload.wikimedia.org/wikipedia/commons/a/a7/Camponotus_flavomarginatus_ant.jpg"
image_media_type = "image/jpeg"
image_data = base64.standard_b64encode(
    httpx.get(image_url, headers={"User-Agent": "AnthropicAPIIntro/1.0"}).content
).decode("utf-8")

message = anthropic.Anthropic().messages.create(
    model="claude-opus-4-6",
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": image_media_type,
                        "data": image_data,
                    },
                },
                {"type": "text", "text": "What is in the above image?"},
            ],
        }
    ],
)
print_msg(message)

{
  "id": "msg_01BnhWM6VgqAWRpFQ4LE5jUq",
  "content": [
    {
      "citations": null,
      "text": "# Ant in Macro Photography\n\nThe image shows a **single ant** captured in stunning **macro (close-up) photography**. The ant appears to be a **black or carpenter ant**, and it's standing on what looks like a concrete or stone surface.\n\nWhat makes this photo particularly striking is the ant's **posture** \u2014 it appears to be rearing up with its head and front legs raised, almost in a defensive or alert stance, with its abdomen (gaster) touching or near the ground. You can clearly see the ant's:\n\n- **Three distinct body segments** (head, thorax, abdomen)\n- **Long antennae**\n- **Six legs** with fine detail\n- **Fine hairs (setae)** on the abdomen\n- **Mandibles** near the head\n\nThe background is beautifully **blurred (bokeh effect)**, with warm tones of brown and pink, keeping the focus entirely on the ant. This is an excellent example of nature/insect macro photography.",
  

##### Option 2

In [8]:
# Option 2: URL-referenced image
message_from_url = anthropic.Anthropic().messages.create(
    model="claude-opus-4-6",
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "url",
                        "url": "https://upload.wikimedia.org/wikipedia/commons/a/a7/Camponotus_flavomarginatus_ant.jpg",
                    },
                },
                {"type": "text", "text": "What is in the above image?"},
            ],
        }
    ],
)
print_msg(message_from_url)

{
  "id": "msg_019fvLSRfBY1dmnyf7xNe2U1",
  "content": [
    {
      "citations": null,
      "text": "# Ant in Macro Photography\n\nThe image shows a **single ant** captured in stunning **macro (close-up) photography**. The ant appears to be a **black or carpenter ant** standing on a rough, concrete-like surface.\n\n## Notable details:\n\n- **Pose**: The ant is in a striking upright position, with its head and front legs raised, almost as if it's rearing up or grooming itself\n- **Body segments**: The three main body parts (head, thorax, and abdomen/gaster) are clearly visible\n- **Features**: You can see its antennae, mandibles, long legs, and fine hairs (setae) on its abdomen\n- **Background**: The background is beautifully blurred (bokeh effect), with warm tones of brown and dark hues\n- **Lighting**: The photograph has excellent lighting that highlights the ant's exoskeleton texture and sheen\n\nThis is a remarkably detailed and well-composed macro photograph that showcases the in